In [1]:
import csv
import networkx as nx
import numpy as np
import pandas as pd
import math
from tqdm import tqdm
import torch
import torch.nn as nn
import torchvision
from sklearn.linear_model import LogisticRegression
from collections import defaultdict
import random

In [2]:
# prepare dataset

f = open('data/train.csv', 'r', encoding='utf-8')
reader = csv.reader(f)
trains = []
for line in reader:
    trains.append(line)
f.close()    
print("trains", len(trains))

f = open('data/node_ingredient.csv', 'r', encoding='utf-8')
reader = csv.reader(f)
node_ingredients = []
for line in reader:
    node_ingredients.append(line)
f.close()    
print("data/node_ingredients", len(node_ingredients))
num_ing = len(node_ingredients)

# f = open('validation_classification_question.csv', 'r', encoding='utf-8')
# reader = csv.reader(f)
# val_cls_q = []
# for line in reader:
#     val_cls_q.append(line)
# f.close()
# print("val_cls_q", len(val_cls_q))

# f = open('validation_classification_answer.csv', 'r', encoding='utf-8')
# reader = csv.reader(f)
# val_cls_a = []
# for line in reader:
#     val_cls_a.append(line)
# f.close()
# print("val_cls_a", len(val_cls_a))

f = open('data/validation_completion_question.csv', 'r', encoding='utf-8')
reader = csv.reader(f)
val_cpt_q = []
for line in reader:
    val_cpt_q.append(line)
f.close()
print("val_cpt_q", len(val_cpt_q))

f = open('data/validation_completion_answer.csv', 'r', encoding='utf-8')
reader = csv.reader(f)
val_cpt_a = []
for line in reader:
    val_cpt_a.append(line)
f.close()
print("val_cpt_a", len(val_cpt_a))

trains 23547
data/node_ingredients 6714
val_cpt_q 7848
val_cpt_a 7848


In [3]:
singles = defaultdict(int)
pairs = defaultdict(int)
triples = defaultdict(int)
quadraples = defaultdict(int)
quintuples = defaultdict(int)

for data in tqdm(trains):
    nodes = [int(node) for node in data[0:-1]]
    nodes = sorted(nodes)
    for i in range(len(nodes)):
        singles[nodes[i]] += 1
            
    for i in range(len(nodes) - 1):
        for j in range(i+1, len(nodes)):
            pairs[(nodes[i], nodes[j])] += 1
            
    for i in range(len(nodes) - 2):
        for j in range(i+1, len(nodes) - 1):
            for k in range(j+1, len(nodes)):
                triples[(nodes[i], nodes[j], nodes[k])] += 1
                
    for i in range(len(nodes) - 3):
        for j in range(i+1, len(nodes) - 2):
            for k in range(j+1, len(nodes) - 1):
                for l in range(k+1, len(nodes)):
                    quadraples[(nodes[i], nodes[j], nodes[k], nodes[l])] += 1
                    
#     for i in range(len(nodes) - 4):
#         for j in range(i+1, len(nodes) - 3):
#             for k in range(j+1, len(nodes) - 2):
#                 for l in range(k+1, len(nodes) - 1):
#                     for t in range(l+1, len(nodes)):
#                         quintuples[(nodes[i], nodes[j], nodes[k], nodes[l], nodes[t])] += 1

print(len(singles))
print(len(pairs))
print(len(triples))
print(len(quadraples))
# print(len(quintuples))

# threshold = 10

# triples = {k:v for k,v in triples.items() if v >= threshold}
# quadraples = {k:v for k,v in quadraples.items() if v >= threshold}
# quintuples = {k:v for k,v in quintuples.items() if v >= threshold}

# print("\n valid")
# print(len(triples))
# print(len(quadraples))
# print(len(quintuples))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 23547/23547 [00:30<00:00, 781.47it/s]

5859
355816
3773815
18745846


In [4]:
def interest(I, j, lamda):
    if len(I) == 4:
        confidence = quintuples[tuple(sorted(I + [j]))] / (quadraples[tuple(sorted(I))] + 0.01)
    elif len(I) == 3:
#         quad = quadraples[tuple(sorted(I + [j]))] if tuple(sorted(I + [j])) in quadraples.keys() else 0
#         trip = triples[tuple(sorted(I))] if tuple(sorted(I)) in triples.keys() else 0.01
#         confidence = quad / trip
        confidence = quadraples[tuple(sorted(I + [j]))] / (triples[tuple(sorted(I))] + 0.01)
    elif len(I) == 2:
        confidence = triples[tuple(sorted(I + [j]))] / (pairs[tuple(sorted(I))] + 0.01)
    else:
        print("error")
    return abs(confidence - lamda * singles[j] / len(trains))

### naive

In [9]:
import time

acc = 0
estimations = []
start = time.time()
for id, data in tqdm(enumerate(val_cpt_q[:10])):
    nodes = [int(node) for node in data]
    target = int(val_cpt_a[id][0])
    interest_list = [0] * num_ing
    len_node = len(nodes)
    for ing in range(num_ing):
        for i in range(len_node - 2):
            for j in range(i+1, len_node - 1):
                for k in range(j+1, len_node):
                    interest_list[ing] += interest([nodes[i], nodes[j], nodes[k]], ing, 0)
                        
    estimation = max(range(len(interest_list)), key=lambda i: interest_list[i])
    
    estimations.append(estimation)
    
    if estimation == target:
        acc += 1

print(time.time() - start)
print("accuracy: ", acc / (id + 1) * 100, "%")
estimations = np.array(estimations)

10it [00:44,  4.42s/it]

44.2277410030365
1
accuracy:  10.0 %


10 iterations take 46 seconds.
It takes too long time!

### efficient

In [ ]:
import time

lamda = 0.25

acc = 0
estimations = []
start = time.time()
for id, data in tqdm(enumerate(val_cpt_q)):
    nodes_temp = [int(node) for node in data]
    nodes = [node for node in nodes_temp if singles[node] > 10]
#     print("nodes: ", len(nodes_temp), len(nodes))
    target = int(val_cpt_a[id][0])
    interest_list = [0] * num_ing
    len_node = len(nodes)
    for ing in range(num_ing):
        if ing in nodes_temp:
            interest_list[ing] = 0
        else:
            for i in range(len_node - 2):
                for j in range(i+1, len_node - 1):
                    for k in range(j+1, len_node):
                        interest_list[ing] += interest([nodes[i], nodes[j], nodes[k]], ing, lamda)
                        
    estimation = max(range(len(interest_list)), key=lambda i: interest_list[i])
#     print(estimation==target, max(interest_list), interest_list[target], nodes_temp, estimation, target)
    estimations.append(estimation)
    
    if estimation == target:
        acc += 1
    
    if id % 100 == 0:
        print(acc / (id+1))

print(time.time() - start)
print("accuracy: ", acc / (id+1) * 100, "%")
estimations = np.array(estimations)

1it [00:00,  2.47it/s]

1.0


101it [06:35,  3.37s/it]

0.15841584158415842


201it [29:30, 15.62s/it]

0.12437810945273632


247it [44:00, 29.73s/it]

10 iterations takes 36.05 seconds

In [8]:
nodelen = defaultdict(int)

for id, data in enumerate(val_cpt_q):
    nodes = [int(node) for node in data]
    nodelen[len(nodes)] += 1

defaultdict(<class 'int'>, {6: 672, 16: 228, 7: 741, 11: 647, 13: 406, 9: 723, 17: 180, 10: 692, 14: 364, 5: 536, 12: 547, 8: 739, 3: 206, 15: 290, 21: 40, 20: 56, 4: 371, 23: 15, 19: 115, 2: 97, 18: 114, 22: 28, 30: 4, 28: 5, 27: 5, 24: 11, 48: 1, 33: 1, 29: 3, 25: 5, 37: 1, 39: 1, 26: 4})


In [ ]:
!sh run.sh

In [10]:
# make val_total.csv
dfs = []
for i in range(40):
    filename = 'results/frequent_items/val_' + str(i * 200) + '.csv'
    df = pd.read_csv(filename, header=None)
    dfs.append(df)
    
total = pd.concat(dfs, ignore_index=True)

print(total.shape)

total.to_csv('results/frequent_items/val_total.csv', sep=',', index=False, header=False)

(7848, 1)


In [7]:
num_937 = 0
for data in val_cpt_a:
    if data[0] == '937':
        num_937 += 1
print(num_937)

321


In [8]:
singles[937]

10683

In [5]:
import time
acc = 0
num_pass = 0
f = open('results/7.txt','w',encoding='utf-8')

for id, data in enumerate(val_cpt_q[7000:]):
    id = id + 7000
    nodes = [int(node) for node in data]
    target = int(val_cpt_a[id][0])

    start = time.time()
    interest_list = [0] * num_ing
    for ing in range(num_ing):
        temp = 0
        for i in range(len(nodes) - 2):
            for j in range(i+1, len(nodes) - 1):
                for k in range(j+1, len(nodes)):
                    interest_list[ing] += interest([nodes[i], nodes[j], nodes[k]], ing)
                    temp += 1
        if temp != 0:
            interest_list[ing] = interest_list[ing] / temp

    estimation = max(range(len(interest_list)), key=lambda i: interest_list[i])
#     print("time :", time.time() - start, ", node lenght: ", len(nodes))
    
    if sum(interest_list) == 0:
        num_pass += 1
    
    if estimation == target:
        acc += 1
        
    print(id, estimation, target, acc)
    f.write("%d,%d,%d\n" % (id, estimation, target))

f.close()


#     if id % 10 == 0 and id > 0:
#         print("accuracy: ", acc / (id + 1) * 100, "%")
    
# print("num val:", len(val_cpt_q))
# print("num_pass: ", num_pass)
# print("accuracy: ", acc / len(val_cpt_q) * 100, "%")

7000 4361 4751 0
7001 937 604 0
7002 937 167 0
7003 937 916 0
7004 4672 4544 0
7005 3978 4672 0
7006 2813 1661 0
7007 937 2221 0
7008 6187 1743 0
7009 937 3969 0
7010 3978 2518 0
7011 2442 6248 0
7012 937 5249 0
7013 937 984 0
7014 5377 6100 0
7015 937 3184 0
7016 6036 3403 0
7017 3554 3147 0
7018 5536 5884 0
7019 937 3228 0
7020 937 5308 0
7021 2122 3978 0
7022 4266 1935 0
7023 937 4454 0
7024 5915 59 0
7025 937 3442 0
7026 1308 1942 0
7027 773 773 1
7028 1953 6187 1
7029 937 6344 1
7030 5648 3152 1
7031 0 6554 1
7032 937 1308 1
7033 2945 1435 1
7034 937 6142 1
7035 1476 6648 1
7036 937 4089 1
7037 937 1179 1
7038 937 2710 1
7039 937 59 1
7040 59 5583 1
7041 937 182 1
7042 937 4799 1
7043 937 2710 1
7044 937 1838 1
7045 5377 585 1
7046 937 207 1
7047 937 6214 1
7048 937 3804 1
7049 937 5884 1
7050 937 937 2
7051 937 937 3
7052 937 1909 3
7053 4544 2464 3
7054 5536 2122 3
7055 167 2805 3
7056 937 4335 3
7057 6187 6187 4
7058 5377 5257 4
7059 937 765 4
7060 6187 5648 4
7061 3978 2414 4


7486 5536 5536 46
7487 2809 4799 46
7488 937 1077 46
7489 937 308 46
7490 937 1396 46
7491 937 992 46
7492 3202 5459 46
7493 59 5204 46
7494 937 4362 46
7495 1816 499 46
7496 937 3630 46
7497 937 1198 46
7498 937 1150 46
7499 2122 332 46
7500 937 3146 46
7501 937 937 47
7502 937 3562 47
7503 937 3652 47
7504 937 2518 47
7505 5536 6126 47
7506 937 3390 47
7507 937 937 48
7508 4799 167 48
7509 0 3978 48
7510 3978 5536 48
7511 5536 5536 49
7512 937 2759 49
7513 937 2790 49
7514 0 5648 49
7515 2945 2938 49
7516 937 3159 49
7517 0 3223 49
7518 5377 1531 49
7519 937 5156 49
7520 5119 3950 49
7521 5536 6456 49
7522 937 5298 49
7523 6187 1198 49
7524 4672 6100 49
7525 937 4038 49
7526 0 3202 49
7527 6187 6100 49
7528 937 937 50
7529 937 6422 50
7530 937 4416 50
7531 937 5970 50
7532 937 1308 50
7533 937 5648 50
7534 791 399 50
7535 5882 5882 51
7536 4038 4038 52
7537 5536 4317 52
7538 937 5377 52
7539 6187 5250 52
7540 937 2274 52
7541 4799 1953 52
7542 5882 3622 52
7543 2122 6352 52
7544 937 

In [ ]:
acc = 0
num_pass = 0

for id, data in tqdm(enumerate(val_cpt_q)):
    nodes = [int(node) for node in data]
    target = int(val_cpt_a[id][0])

    interest_list = [0] * num_ing
    for ing in range(num_ing):
        temp = 0
        for i in range(len(nodes) - 3):
            for j in range(i+1, len(nodes) - 2):
                for k in range(j+1, len(nodes) - 1):
                    for l in range(k+1, len(nodes)):
                        interest_list[ing] += interest([nodes[i], nodes[j], nodes[k], nodes[l]], ing)
                        temp += 1
        if temp != 0:
            interest_list[ing] = interest_list[ing] / temp

    estimation = max(range(len(interest_list)), key=lambda i: interest_list[i])
    
    if sum(interest_list) == 0:
        num_pass += 1
    
    if estimation == target:
        acc += 1
        
    if id % 10 == 0 and id > 0:
        print("accuracy: ", acc / (id + 1) * 100, "%")
    
print("num val:", len(val_cpt_q))
print("num_pass: ", num_pass)
print("accuracy: ", acc / len(val_cpt_q) * 100, "%")

11it [01:58,  7.51s/it]

accuracy:  0.0 %


21it [03:03,  4.07s/it]

accuracy:  0.0 %


31it [03:44,  2.97s/it]

...

181it [40:09, 18.32s/it]

accuracy:  6.629834254143646 %


191it [41:06,  4.21s/it]

accuracy:  6.282722513089005 %


194it [41:20,  4.42s/it]

In [11]:
acc = 0
num = 0
for i in range(8):
    f = open("results/" + str(i) + ".txt", 'r')
    for line in f.readlines():
        id, estimation, target = line.strip().split(',')
        num += 1
        if estimation == target:
            acc += 1
    f.close()
print(acc / num * 100)

9.097859327217126


In [ ]:
p